# Отчёт по соревнованию 1: Предскажите вероятность победы команды в Dota 2

## Попов Артём Сергеевич ММП ВМК МГУ 2016

### Описание задания:
Dota 2 — многопользовательская компьютерная игра жанра MOBA. Игроки играют между собой матчи. В каждом матче, как правило, участвует 10 человек. Матчи формируются из живой очереди, с учётом уровня игры всех игроков. Перед началом игры игроки автоматически разделяются на две команды по пять человек. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды уничтожить главное здание базы противника - трон.

Участникам соревнования предлагалась обучающая выборка из матчей, для которых известны все события матча и его результат. В тестовой выборке матчей история обрезана на первых 5 минутах. Участникам сореванования предлагалось предсказать для тестовых матчей вероятность победы команды Radiant.

### Описание данных

Было несколько разных данных, которые можно было применить для обучения:
* Сформированные обучающая и тестовая выборки матчей
* "Сырые" матчи - словари, из которых можно было взять дополнительную информацию
* Таблица характеристик и ролей героев, сделанные по данным с официального сайта Dota2 (дополнительные данные, сформированные Эмилем Каюмовым и мной)

### Описание решения (словесное описание и код)

(Внимание! В таком виде код никогда не запускался. Во время соревнования код был разбит по разным модулям (для удобства).)

#### Подключение необходимых библиотек

Основные библиотеки для работы с массивами:

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as scpst

Функции для предобработки данных:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

Функции для измерения качества модели:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

Функции, реализующие алгоритмы машинного обучения:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb

Функции для работы с "сырыми матчами":

In [ ]:
import json
import bz2

#### Подготовка данных

##### Мешки слов по предметам и способностям

В информации, содержащейся в "сырых" матчах и неучтённой в обучающей выборке, показались важными два компонента:
1. Информация о предметах героев (в обучающей выборке была только информация о количестве предметов)
2. Информация о выбранных улучшениях способностей героев

По этим данным были составлены мешки предметов и мешки способностей для каждой из команд (у каждой команды свой мешок).

In [ ]:
def filter_events(events, time_point=60*5):
    return [event for event in events if event['time'] <= time_point]

In [ ]:
def get_bags(name_file):
    item_data = pd.DataFrame(np.zeros((97230, 11)),
                             columns=['match_id'] + ['r' + str(i) + '_items' for i in range(1, 6)] +
                             ['d' + str(i) + '_items' for i in range(6, 11)])
    ability_data = pd.DataFrame(np.zeros((97230, 11)),
                                columns=['match_id'] + ['r' + str(i) + '_ability' for i in range(1, 6)] +
                                ['d' + str(i) + '_ability' for i in range(6, 11)])
    j = -1
    with bz2.BZ2File(name_file) as matches_file:
        for line in matches_file:
            j += 1
            match = json.loads(line)

            item_data['match_id'].iloc[j] = match['match_id']
            ability_data['match_id'].iloc[j] = match['match_id']

            for i in range(0, 10):
                if i < 5: 
                    item_data['r' + str(i + 1) + '_items'].iloc[j] = ' '.join(map(lambda x: str(x['item_id']),
                                                                    filter_events(match['players'][i]['purchase_log'])))

                    ability_data['r' + str(i + 1) + '_ability'].iloc[j] = ' '.join(map(lambda x: str(x['ability']),
                                                                    filter_events(match['players'][i]['ability_upgrades'])))

                else:
                    item_data['d' + str(i + 1) + '_items'].iloc[j] = ' '.join(map(lambda x: str(x['item_id']),
                                                                    filter_events(match['players'][i]['purchase_log'])))

                    ability_data['d' + str(i + 1) + '_ability'].iloc[j] = ' '.join(map(lambda x: str(x['ability']),
                                                                    filter_events(match['players'][i]['ability_upgrades'])))


            if j % 10000 == 0:
                print(j)
    item_data = item_data.sort(['match_id'])
    ability_data = ability_data.sort(['match_id'])
    
    item_data.loc[np.where(item_data.isnull())] = 'nan'
    ability_data.loc[np.where(ability_data.isnull())] = 'nan'
    
    item_data['r_items'] = item_data['r1_items'] + ' ' + item_data['r2_items'] + ' ' + item_data['r3_items'] + ' ' +
                           item_data['r4_items'] + ' ' + item_data['r5_items']
    item_data['d_items'] = item_data['d6_items'] + ' '+ item_data['d7_items'] + ' ' + item_data['d8_items'] + ' ' +
                           item_data['d9_items'] + ' ' + item_data['d10_items']
    item_data = item_data[['match_id', 'r_items', 'd_items']]
    
    
    ability_data['r_ability'] = ability_data['r1_ability'] + ' ' + ability_data['r2_ability'] + ' ' + ability_data['r3_ability'] +
                                ' ' + ability_data['r4_ability'] + ' ' + ability_data['r5_ability']
    ability_data['d_ability'] = ability_data['d6_ability'] + ' ' + ability_data['d7_ability'] + ' ' + ability_data['d8_ability'] +
                                ' ' + ability_data['d9_ability'] + ' ' + ability_data['d10_ability']
    ability_data = ability_data[['match_id', 'r_ability', 'd_ability']]

    return item_data, ability_data

In [ ]:
item_data, ability_data = get_bags('data/matches.jsonlines.bz2')
item_data_test, ability_data_test = get_bags('data/matches_test.jsonlines.bz2')

Мешок по предметам:

In [ ]:
count_vec = CountVectorizer(min_df=1)
count_vec.fit(item_data['r_items'])
r_train = count_vec.transform(item_data['r_items'])
r_test = count_vec.transform(item_data_test['r_items'])

count_vec.fit(item_data['d_items'])
d_train = count_vec.transform(item_data['d_items'])
d_test = count_vec.transform(item_data_test['d_items'])

In [ ]:
np.save('data/items_train.npy', np.hstack((r_train.toarray(), d_train.toarray())))
np.save('data/items_test.npy', np.hstack((r_test.toarray(), d_test.toarray())))

Мешок по способностям:

In [ ]:
count_vec = CountVectorizer(min_df=1)
count_vec.fit(ability_data['r_ability'])
r_train = count_vec.transform(ability_data['r_ability'])
r_test = count_vec.transform(ability_data_test['r_ability'])

count_vec.fit(ability_data['d_ability'])
d_train = count_vec.transform(ability_data['d_ability'])
d_test = count_vec.transform(ability_data_test['d_ability'])

In [ ]:
np.save('data/ability_train.npy', np.hstack((r_train.toarray(), d_train.toarray())))

In [ ]:
np.save('data/ability_test.npy', np.hstack((r_test.toarray(), d_test.toarray())))

Функции, для последующего добавления мешков:

In [ ]:
def add_item_bag(data_train, data_test):
    item_bag_train = np.load('data/items_train.npy')
    item_bag_test = np.load('data/items_test.npy')
    data_train = pd.concat([data_train,
                            pd.DataFrame(item_bag_train, columns = ['it_bag' + str(i) for i in range(item_bag_train.shape[1])])],
                           axis=1)
    data_test = pd.concat([data_test,
                           pd.DataFrame(item_bag_test, columns = ['it_bag' + str(i) for i in range(item_bag_train.shape[1])])],
                          axis=1)
    return data_train, data_test
    
def add_ability_bag(data_train, data_test):
    ability_bag_train = np.load('data/ability_train.npy')
    ability_bag_test = np.load('data/ability_test.npy')
    data_train = pd.concat([data_train,
                            pd.DataFrame(ability_bag_train, 
                                         columns = ['ab_bag' + str(i) for i in range(ability_bag_train.shape[1])])],
                           axis=1)
    data_test = pd.concat([data_test,
                           pd.DataFrame(ability_bag_test,
                                        columns = ['ab_bag' + str(i) for i in range(ability_bag_train.shape[1])])],
                          axis=1)
    return data_train, data_test


##### Синергия и антисинергия героев

В статье http://cseweb.ucsd.edu/~jmcauley/cse255/reports/fa15/018.pdf предложен способ учёта взаимодействия между парами героев. Авторы статьи предлагают сделать два дополнительных признака: синергию и антисинергию.

Необходимо подсчитать для всех героев по обучающей выборке характеристики:
* $S_{ij} $ - количество раз, когда i-ый и j-ый герой были в одной команде и победили
* $C_{ij} $ - количество раз, когда команда, в которой был i-ый герой, победила команду, в которой был j-ый герой

Тогда:
* Синергия: $\; \sum_{i, j \in radiant}S_{ij} - \sum_{i, j \in dire}S_{ij}$
* Антисинергия: $\; \sum_{i, j \in dire}C_{ij} - \sum_{i, j \in dire}C_{ij}$

Если использовать этот подход, эти два признака приводят к сильному переобучению модели. 
Можно воспользоваться подходом, похожим на тот, что применяется при использовании счётчиков. 
Разобьём выборку на несколько фолдов, для каждого фолда синергия и антисинергия считается по всем остальным фолдам.
Синергия и антисинергия тестовой выборке считается по обучающей выборке. Выбранное число фолдов - 30.

In [ ]:
def calc_rating(data):
    N = 113 # heroes
    # calculate each hero-pair synergy and antisynergy
    synergy = np.zeros((N,N))     # sum of wins in matches played together
    antisynergy = np.zeros((N,N)) # sum of wins when played against
    matchcounts = np.zeros((N,N)) # count of matches played together
    matchcounta = np.zeros((N,N)) # count of matches played against
    
    for match_counter, match_id in enumerate(data.index):
        #synergy when both heroes in win team
        winteam = 'r' if data.ix[match_id, 'radiant_win'] == 1 else 'd'
        looseteam = 'd' if winteam =='r' else 'r'
        pind     = [0] *5 #player indexes    
        antipind = [0] *5 # looser indicies
        # get indexes of players in each tem
        for i in range(5):
            pind[i] = data.ix[match_id, winteam+'%d_hero'%(i+1)]-1
        for i in range(5):
            antipind[i] = data.ix[match_id, looseteam+'%d_hero'%(i+1)]-1
        # accumulate synergy of pairs
        for i in range(5):
            for j in range(i+1,5):
                synergy[pind[i], pind[j]] +=1
                synergy[pind[j], pind[i]] +=1
        # accumulate match counter for playing together
        for i in range(5):
            for j in range(5):
                matchcounts[pind[i], pind[j]] +=1 #together and win
                matchcounts[antipind[i], antipind[j]] +=1 # together and loose
                
        #antisynergy when hero i in winteam while hero j in loose team
        for i in range(5):
            for j in range(5):
                antisynergy[pind[i], antipind[j]] +=1
                matchcounta[pind[i], antipind[j]] +=1
                matchcounta[antipind[j], pind[i]] +=1
            
    # normalize
    synergyrate = np.zeros((N,N))
    antisynergyrate = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if matchcounts[i,j] !=0:
                synergyrate[i,j] = synergy[i,j]/matchcounts[i,j] 
            else:
                synergyrate[i,j] = 0.5  
            if matchcounta[i,j] !=0:
                antisynergyrate[i,j] = antisynergy[i,j]/ matchcounta[i,j]
            else:
                antisynergyrate[i,j] = 0.5      

    return synergyrate, antisynergyrate


def add_synergy(data, synergyrate, antisynergyrate):
    # calculate aggreagtes synergy and antisyn    
    syn = np.zeros(len(data))
    antisyn = np.zeros(len(data))
    for match_counter, match_id in enumerate(data.index):
        rind = [0] *5 #radiant indicies    
        dind = [0] *5 # dire indicies
        # get indexes of players in each team
        for i in range(5):
            rind[i] = data.ix[match_id, 'r%d_hero'%(i+1)]-1
        for i in range(5):
            dind[i] = data.ix[match_id, 'd%d_hero'%(i+1)]-1
        # accumulate synergy of radiants minus synergy of dires
        # + radiants synergy
        for i in range(5):
            for j in range(i+1,5):
                syn[match_counter] += synergyrate[rind[i], rind[j]]
        # - dire synergy
        for i in range(5):
            for j in range(i+1,5):
                syn[match_counter] -= synergyrate[dind[i], dind[j]]
        # accumulate antisynergy
        for i in range(5):
            for j in range(5):
                antisyn[match_counter] += antisynergyrate[rind[i], dind[j]] 
                
    data['synergy'] = syn
    data['antisynergy'] = antisyn
    return data

In [ ]:
# preprocess
data_train = pd.read_csv('data/features.csv')
data_train.fillna(0, inplace=True)

data_test = pd.read_csv('data/features_test.csv')
data_test.fillna(0, inplace=True)

### для теста
synergyrate, antisynergyrate = calc_rating(data_train)
data_test = add_synergy(data_test, synergyrate, antisynergyrate)

### для трэйна
data_train['synergy'] = 0
data_train['antisynergy'] = 0

cv = KFold(data_train.shape[0], n_folds=30)

for ind_train, ind_test in cv:
    synergyrate, antisynergyrate = calc_rating(data_train.loc[ind_train, :])
    data_train.loc[ind_test, :] = add_synergy(data_train.loc[ind_test, :], synergyrate, antisynergyrate)
    
data_train.to_csv('data/new_features.csv')
data_test.to_csv('data/new_features_test.csv')

##### Роли героев

Из всей таблицы с дополнительными данными были выбраны столбцы отвечающие принадлежности героев к:
* определённой роли в Dota2 (Carry, Nuker и т.д.)
* типу атаки (ближний или дальний бой)
* главной характеристики (сила, ловкость, интеллект)

Для каждой команды были подсчитаны статистики: сколько в этой команде героев какого типа (т.е. сколько в команде Carry, сколько героев ближнего боя, сколько силачей и т.д.)

In [ ]:
data_hero = pd.read_csv('heroes_db.csv', index_col=0)
data = pd.read_csv('data/new_features.csv')
test = pd.read_csv('data/new_features_test.csv')

In [ ]:
def add_super_new_features(data, data_hero):
    for string in [u'Carry', u'Disabler', u'Lane_support', u'Initiator', u'Jungler', u'Support', u'Durable', u'Pusher',
                   u'Nuker', u'Escape', u'Melee', u'Ranged', u'Strength', u'Intelligence', u'Agility']:
        for team in ['r', 'd']:
            num_hero = [team + str(i) + '_hero' for i in range(2, 6)]
            data[team + '_' + string] = data_hero[string][data[team + '1_hero']].values
            for str_hero in num_hero:
                data[team + '_' + string] += data_hero[string][data[str_hero]].values
                        
    return data

In [ ]:
data = add_super_new_features(data, data_hero)
test = add_super_new_features(test, data_hero)

data.to_csv('data_with_super_features.csv', index='Unnamed: 0')
test.to_csv('test_with_super_features.csv', index='Unnamed: 0')

##### One-hot-encoding героев
Для успешной работы алгоритма необходимо закодировать информацию о том, какие герои были в каких командах. Был выбран способ предложенный в https://github.com/esokolov/ml-course-msu/blob/master/ML15-spring/contests/contest01-dota/contest01-dota-statement.ipynb: one-hot-encoding героев.

In [ ]:
def encoding(encode, data):
    arr1 = encode.transform(data.r1_hero.values.reshape(-1, 1)).toarray()
    arr2 = encode.transform(data.r2_hero.values.reshape(-1, 1)).toarray()
    arr3 = encode.transform(data.r3_hero.values.reshape(-1, 1)).toarray()
    arr4 = encode.transform(data.r4_hero.values.reshape(-1, 1)).toarray()
    arr5 = encode.transform(data.r5_hero.values.reshape(-1, 1)).toarray()
    arr = arr1 + arr2 + arr3 + arr4 + arr5
    
    
    arr1 = encode.transform(data.d1_hero.values.reshape(-1, 1)).toarray()
    arr2 = encode.transform(data.d2_hero.values.reshape(-1, 1)).toarray()
    arr3 = encode.transform(data.d3_hero.values.reshape(-1, 1)).toarray()
    arr4 = encode.transform(data.d4_hero.values.reshape(-1, 1)).toarray()
    arr5 = encode.transform(data.d5_hero.values.reshape(-1, 1)).toarray()
    arr = arr - (arr1 + arr2 + arr3 + arr4 + arr5)
    
    dataframe = pd.DataFrame(arr, columns=[str('hero_') + str(n) for n in range(arr.shape[1])])
    dataframe.index.name = 'match_id'
    
    return dataframe

##### Изменение основных признаков

Для улучшения модели был использован следующий приём: признаки, относящиеся к героям, были отсортированы внутри каждой команды.
Было сделано one-hot кодирование для признака lobby type. Был создан дополнительный признак отношение смертей к убийствам. Было сделано небольшое преобразование над признаками, отвечающими количеству смертей.

In [ ]:
def delete_missing_values(imputer, data):
    temp = imputer.transform(data)
    data.iloc[:, :] = temp
    return data

def data_prepare(data_train, data_test):
    new_data_train = pd.DataFrame()
    new_data_test = pd.DataFrame()
    
    y_train = (data_train.radiant_win.ravel() - 0.5) * 2
    
    index_answer = ['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire',
                    'barracks_status_dire', 'barracks_status_radiant']
    index_heroes = ['r' + str(i) + '_hero' for i in range(1, 6)]
    index_other = ['match_id', 'start_time']
    
    
    ### missing values
    imp = Imputer(strategy='median')
    imp.fit(data_train.drop(index_answer, axis=1))
    
    new_data_train = delete_missing_values(imp, data_train.drop(index_answer, axis=1))
    new_data_test = delete_missing_values(imp, data_test)
    
    ### One-Hot-Encoding
    encoder = OneHotEncoder()
    encoder.fit(data_train.r1_hero.values.reshape(-1, 1))
    #train_encode = encoding(encoder, data_train)
    new_data_train = pd.concat([new_data_train, encoding(encoder, data_train)], axis=1)
    new_data_test = pd.concat([new_data_test, encoding(encoder, data_test)], axis=1)
    
    
    encoder2 = OneHotEncoder()
    encoder2.fit(data_train.lobby_type.values.reshape(-1, 1))
    
    new_data_train = pd.concat([new_data_train,
                                pd.DataFrame(encoder2.transform(data_train.lobby_type.values.reshape(-1, 1)).toarray())],
                               axis=1)
    new_data_test = pd.concat([new_data_test,
                               pd.DataFrame(encoder2.transform(data_test.lobby_type.values.reshape(-1, 1)).toarray())],
                              axis=1)

    ### исключаем плохие признаки
    
    index_total = index_heroes + index_other
    new_data_train = new_data_train.drop(index_total, axis=1)
    new_data_test = new_data_test.drop(index_total, axis=1)
    
    
    ### добавление фичей
    new_data_train = add_features(new_data_train)
    new_data_test = add_features(new_data_test)
    
    ### скейлинг
    scale = StandardScaler()
    scale.fit(new_data_train)
    new_data_train = pd.DataFrame(scale.transform(new_data_train), columns=new_data_train.columns)
    new_data_test = pd.DataFrame(scale.transform(new_data_test), columns=new_data_train.columns)
    return new_data_train.drop(new_data_train.columns[:76], axis=1),
           y_train, new_data_test.drop(new_data_train.columns[:76], axis=1)

def add_features(data):
    for i in range(1, 6):
        data['r' + str(i) + '_prob_ratio'] = (data['r' + str(i) + '_deaths']) / (data['r' + str(i) + '_kills'] + 1)
        data['d' + str(i) + '_prob_ratio'] = (data['d' + str(i) + '_deaths']) / (data['d' + str(i) + '_kills'] + 1)
                
    for string_feat in ['xp','level', 'gold', 'lh', 'kills', 'deaths', 'items']:
        new_feat = np.zeros(data.shape[0])
        r_feat = ['r' + str(i) + '_' + string_feat for i in range(1, 6)]
        d_feat = ['d' + str(i) + '_' + string_feat for i in range(1, 6)]
        
        data['r_' + string_feat + '_total'] = np.sum(data[r_feat], axis=1)
        data['d_' + string_feat + '_total'] = np.sum(data[d_feat], axis=1)
        
        
        temp1 = np.sort(data[r_feat], axis=1)
        temp2 = np.sort(data[d_feat], axis=1)
        
        for i in range(1, 6):
            data['r_' + string_feat + '_max' + str(i)] = temp1[:, i - 1]
            data['d_' + string_feat + '_max' + str(i)] = temp2[:, i - 1]
            
            if string_feat == 'deaths':
                data['r_' + string_feat + '_max' + str(i)] = temp1[:, i - 1] ** 1.1
                data['d_' + string_feat + '_max' + str(i)] = temp2[:, i - 1] ** 1.1
        
    return data

#### Применение алгоритмов
Для решения поставленной задачи был выбран алгоритм - логистическая регрессия. На финальном этапе решения задачи было решено составить ансамбль из логистической регрессии и градиентного бустинга. Для бустинга использовалась калибировка вероятностей.

##### Логистическая регрессия

Для логистической регрессии были использованы следующие наборы признаков:

* преобразованная обучающая выборка
* синергия и антисинергия
* мешок предметов

In [ ]:
train_path = 'data/new_features.csv'
data_train = pd.read_csv(train_path)

test_path = 'data/new_features_test.csv'
data_test = pd.read_csv(test_path)

In [ ]:
X_train, y_train, X_test = data_prepare(data_train, data_test)
X_train, X_test = add_item_bag(X_train, X_test)

In [ ]:
clf1 = LogisticRegression()
clf1.fit(X_train, y_train)

In [ ]:
pred1 = clf1.predict_proba(X_test)[:, 1]
preds_csv = pd.DataFrame({"match_id": data_test['match_id'], "radiant_win": pred1})
preds_csv = preds_csv.set_index("match_id")
preds_csv.to_csv('preds/log_reg_predictions.csv')

##### Градиентный бустинг

Для градиентного бустинга были использованы следующие наборы признаков:

* преобразованная обучающая выборка
* синергия и антисинергия
* мешок предметов
* мешок способностей
* роли героев (доп. данные)

In [ ]:
train_path = 'data_with_super_features.csv'
data_train = pd.read_csv(train_path)

test_path = 'test_with_super_features.csv'
data_test = pd.read_csv(test_path)

In [ ]:
X_train, y_train, X_test = data_prepare(data_train, data_test)
X_train, X_test = add_item_bag(X_train, X_test)
X_train, X_test = add_ability_bag(X_train, X_test)

In [ ]:
gbm = CalibratedClassifierCV(base_estimator=xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.2, nthread=2))
gbm.fit(X_train, y_train)

In [ ]:
pred1 = gbm.predict_proba(X_test)[:, 1]
preds_csv = pd.DataFrame({"match_id": data_test['match_id'], "radiant_win": pred1})
preds_csv = preds_csv.set_index("match_id")
preds_csv.to_csv('preds/gbm_all.csv')

##### Ансамбль

Для итоговой модели предсказания были усреднены с весами 0.65 (лог. регрессия) и 0.35(бустинг).

In [ ]:
preds_csv = pd.DataFrame({"match_id": data_test['match_id'], "radiant_win": 0.65 * pred1 + 0.35 * pred2})
preds_csv = preds_csv.set_index("match_id")
preds_csv.to_csv('preds/ansamble_3.csv')

### Результаты

* 0.76490 - public (50% выборки) 
* 0.76455 - private (50% выборки)
* 0.76474 - вся выборка

В целом, можно сказать, что задача труднорешаема. Пять первых минут матча Dota дают очень мало информации об игроках, чтобы сделать хорошее предсказание. ROC-AUC в 0.75 достигается только с помощью one-hot-encoding героев, остальные навороты (а их было немало) позволили улучшить начальный результат примерно на 1.5.